In [8]:
using Pkg; Pkg.activate("."); 
using StatsBase
using NPZ
using MUST
using PyPlot
using Interpolations
using PyCall
using KernelDensity
using DelimitedFiles
using Printf
using TSO

#import PyCall.rc
#rc("font", size=17)

MUST.@import_dispatch "/u/peitner/DISPATCH/dispatch2"

MUST.@get_help visual
pl = pyimport("matplotlib.pylab");

  Activating environment at `/cobra/ptmp/peitner/model_grid/MUST.jl/examples/solar_model/Project.toml`


In [5]:
folders = [ MUST.@in_dispatch("data_tmps/DIS_MARCS_v0.4.4_TDZ2")]
snaps   = [93]
labels  = ["MARCS (4 bins) - cooler adiabat"]

model, model_t = [first(MUST.Boxes(folder1, snaps=[snaps[i]])) for (i,folder1) in enumerate(folders)], 
                 [last(MUST.Boxes(folder1, snaps=[snaps[i]])) for (i,folder1) in enumerate(folders)];

In [4]:
eos_path = "/u/peitner/Turbospectrum/opacity_tables/tests/DIS_MARCS_v0.5.1"
eos = reload(SqEoS, joinpath(eos_path, "eos.hdf5"))
opa = reload(SqOpacity, joinpath(eos_path, "binned_opacities.hdf5"));

In [6]:
m = last(model);

In [10]:
κ = lookup(eos, opa, :κ,   log.(m[:d]), log.(m[:T]), 1)
S = lookup(eos, opa, :src, log.(m[:d]), log.(m[:T]), 1);

In [12]:
size(κ)

(200, 200, 100)

In [11]:
npzwrite("cube.npz", Dict("opacity" => κ, "source_function" => S, "Temperature" => m[:T]))